In [21]:
import os

import pandas as pd
import numpy as np
import pandas_gbq

from google.cloud import bigquery
from google.oauth2 import service_account

import sqlite3

In [22]:
#These first two values will be different on your machine.
service_path = ""
service_file = 'umt-msba-gg-key.json'

gbq_proj_id = 'umt-msba'

#And this should stay the same.
private_key = service_path + service_file
#Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(private_key)
#And finally we establish our connection.
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)
for item in client.list_datasets() :
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


In [23]:
query="""

SELECT
 DATE(datetime) AS formatted_date,
 EXTRACT(hour FROM TIMESTAMP(datetime) AS hour,
 ROUND(SUM(total), 2) AS spend,
 COUNT(DISTINCT(CONCAT(EXTRACT(date FROM datetime),register_no,emp_no,trans_no))) AS trans,
 SUM(CASE WHEN trans_status in ("R", "V") THEN -1 ELSE 1 END) AS items
FROM 'umt-msba.transactions.transArchive_*'
WHERE department not in (0, 15) AND
(trans_status is null OR
trans_status in ("R","V", " "))
GROUP BY formatted_date, hour 
ORDER BY formatted_date, hour;

"""

In [24]:
conn = sqlite3.connect("reporting.db")

In [ ]:
holder = pandas_gbq.read_gbq(query,project_id=gbq_proj_id,credentials=credentials)

In [ ]:
holder.head()